In [2]:
# Ensure project root is on sys.path so sibling packages are importable
import os
import sys

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print("Added to sys.path:", project_root)


Added to sys.path: c:\Users\betus\Desktop\all_projects\LLM_Projects


In [3]:
# imports
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from WebScraper_GPT_Summarizer.scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [10]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [4]:
links = fetch_website_links("https://www.geeksforgeeks.org/dsa/dsa-tutorial-learn-data-structures-and-algorithms/")
links

['#main',
 'https://www.geeksforgeeks.org/',
 'https://www.geeksforgeeks.org/python/python-programming-language-tutorial/',
 'https://www.geeksforgeeks.org/java/java/',
 'https://www.geeksforgeeks.org/learn-data-structures-and-algorithms-dsa-tutorial/',
 'https://www.geeksforgeeks.org/ai-ml-and-data-science-tutorial-learn-ai-ml-and-data-science/',
 'https://www.geeksforgeeks.org/interview-corner/',
 'https://www.geeksforgeeks.org/programming-language-tutorials/',
 'https://www.geeksforgeeks.org/web-technology/',
 'https://www.geeksforgeeks.org/articles-on-computer-science-subjects-gq/',
 'https://www.geeksforgeeks.org/devops/devops-tutorial/',
 'https://www.geeksforgeeks.org/websites-apps/software-and-tools-a-to-z-list/',
 'https://www.geeksforgeeks.org/geeksforgeeks-school/',
 'https://www.geeksforgeeks.org/geeksforgeeks-practice-best-online-coding-platform/',
 'https://www.geeksforgeeks.org/courses/category/dsa-placements',
 'https://www.geeksforgeeks.org/courses/category/machine-lea

## Step 1: Extract Links From Website

I will use GPT-5-nano for figure out which links are relevant.
I will use "one shot prompting" in which I provide an example of how it should respond in the prompt.


In [5]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):
"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [7]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edward

In [8]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [11]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'company page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'linkedin profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'twitter profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'facebook page',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

In [12]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [13]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gpt-5-nano
Found 7 relevant links


{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'company page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'blog', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'linkedin', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'twitter', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'facebook', 'url': 'https://www.facebook.com/edward.donner.52'}]}

In [14]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 11 relevant links


{'links': [{'type': 'careers page',
   'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'brand page', 'url': 'https://huggingface.co/brand'},
  {'type': 'documentation hub', 'url': 'https://huggingface.co/docs'},
  {'type': 'endpoints product page',
   'url': 'https://endpoints.huggingface.co'},
  {'type': 'community / discussion forum',
   'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub profile', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter profile', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn profile',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Home page', 'url': 'https://huggingface.co/'}]}

## Step 2: Make the Brochure

In [15]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [17]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 11 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
deepseek-ai/DeepSeek-OCR
Updated
5 days ago
•
1.18M
•
2.2k
MiniMaxAI/MiniMax-M2
Updated
26 minutes ago
•
132k
•
695
tencent/HunyuanWorld-Mirror
Updated
4 days ago
•
17.5k
•
385
PaddlePaddle/PaddleOCR-VL
Updated
about 3 hours ago
•
20.4k
•
1.15k
meituan-longcat/LongCat-Video
Updated
about 10 hours ago
•
466
•
191
Browse 1M+ models
Spaces
Running
on
Zero
335
335
DeepSeek OCR Demo
🆘
An interactive demo for the DeepSeek-OCR model.
Running
15.5k
15.5k
DeepSite v3
🐳
Generate any application by Vibe Coding
Running
2.14k
2.14k
Wan2.2 Animate

In [18]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [19]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [20]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 9 relevant links


"\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\ndeepseek-ai/DeepSeek-OCR\nUpdated\n5 days ago\n•\n1.18M\n•\n2.2k\nMiniMaxAI/MiniMax-M2\nUpdated\n42 minutes ago\n•\n132k\n•\n696\ntencent/HunyuanWorld-Mirror\nUpdated\n4 days ago\n•\n17.5k\n•\n386\nPaddlePaddle/PaddleOCR-VL\nUpdated\nabout 3 hours ago\n•\n20.4k\n•\n1.15k\nmeituan-longcat/LongCat-Video\nUpdated\nabout 10 hours ago\n•\n466\n•\n191\nBrowse 1M+ models\nSpaces\nRunning\non\nZero

In [23]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [24]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 11 relevant links


# Hugging Face - The AI Community Building the Future

---

## About Hugging Face
Hugging Face is a leading collaboration platform for the machine learning (ML) community, dedicated to building an open and ethical AI future together. It serves as a central hub where ML engineers, scientists, and end users can share, explore, discover, and experiment with open-source models, datasets, and applications. With a fast-growing community and some of the most popular open-source ML libraries and tools, Hugging Face stands at the forefront of the AI revolution.

---

## What Hugging Face Offers

### Models
- Browse over **1 million models** in diverse modalities including text, image, video, audio, and even 3D.
- Popular models like DeepSeek-OCR, MiniMax-M2, PaddleOCR-VL, and more.
- Offers an interactive demo space to test and experiment with models instantly.

### Datasets
- Access to over **250,000 datasets** for training, evaluation, and building ML applications.
- Examples include datasets for autonomous vehicles, web text corpora, and computer vision.

### Spaces
- Collaborative ML applications hosted on the platform, with thousands of interactive demos and tools.
- Enables users to create, share, and explore AI-powered applications easily.

### Community & Collaboration
- A vibrant, growing ML community focused on open collaboration and knowledge sharing.
- Allows users to build their machine learning portfolio by sharing models and datasets.
- Supports discovery and experimentation for both new and seasoned ML practitioners.

### Enterprise & Team Solutions
- Offers advanced platform features tailored for teams and enterprises, including:
  - Enterprise-grade security with Single Sign-On (SSO) integration.
  - Granular access control with resource groups, token management, and audit logs.
  - Advanced analytics dashboards to monitor usage and inform decisions.
  - Scalable compute options including enhanced ZeroGPU quotas for efficient model deployment.
  - Private storage and dataset viewing for secure internal collaboration.
  - Flexible subscription plans starting at $20/user/month for teams, with custom contracts for enterprises.

---

## Company Culture
- Hugging Face fosters an open, inclusive, and innovative community culture.
- Committed to building ethical, transparent AI technologies that empower developers and researchers worldwide.
- Strong emphasis on collaboration, continuous learning, and sharing to accelerate AI advancements.
- Supports contributors from various backgrounds—research, industry, and academia—encouraging diversity and creativity.

---

## Careers at Hugging Face
- Hugging Face is growing rapidly and actively recruiting talented individuals passionate about AI and machine learning.
- Offers exciting career opportunities in research, engineering, product development, and community management.
- Working at Hugging Face means joining a forward-thinking company that values open-source contributions and ethical AI development.
- Candidates can expect a dynamic environment with opportunities to work on cutting-edge ML technologies and impact the global AI community.

---

## Why Choose Hugging Face?

- **Largest and most active ML Hub**: Millions of models, datasets, and applications to accelerate your AI projects.
- **Trusted by Researchers and Enterprises**: With secure, scalable, and enterprise-grade solutions tailored for diverse organizational needs.
- **Collaborative and Open Source**: Power your work with community support and transparent, accessible resources.
- **Cutting-edge Innovation**: Access the latest tools and research from a dedicated science team exploring the edge of AI technology.

---

## Get Started Today
Join the Hugging Face community to build, share, and discover the future of machine learning.

- Sign up at: [huggingface.co](https://huggingface.co)
- Explore models, datasets, and apps
- Build your ML portfolio and collaborate globally
- Unlock powerful enterprise solutions to scale your AI initiatives

---

*Hugging Face – The AI community building the future.*

## Step 3: Typewriter Animation 

In [25]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [26]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 7 relevant links


# Hugging Face Brochure

---

## About Hugging Face

**Hugging Face** is the AI community building the future of machine learning. It serves as a vibrant collaboration platform where machine learning practitioners—from beginners to world-class researchers—can create, discover, share, and experiment with models, datasets, and AI applications. The platform empowers a global community to accelerate the development of open, ethical, and cutting-edge AI technology.

---

## What We Offer

- **Models:** Access and contribute to over 1 million machine learning models across multiple modalities including text, image, video, audio, and 3D.
- **Datasets:** Explore a vast repository with 250,000+ datasets that fuel ML innovation.
- **Spaces:** Run and share interactive ML applications (over 400,000+) that demonstrate AI capabilities in real-time.
- **Community:** Join one of the largest and fastest-growing ML communities to collaborate, learn, and contribute.
- **Enterprise Solutions:** Tailored paid compute and enterprise-grade platforms with enhanced security, access controls, and collaboration tools designed for teams.

---

## Why Choose Hugging Face?

- **Open Source at Core:** Utilize the powerful Hugging Face open-source stack to move faster in your AI projects.
- **Multi-Modality Support:** Work effortlessly across text, images, video, audio, and 3D AI models.
- **Portfolio Building:** Showcase your machine learning projects and build a professional profile visible across the community.
- **Interactive ML Applications:** Deploy and experiment with AI apps built by the community to see machine learning in action.
- **Enterprise-Grade Security:** For businesses and teams, enjoy advanced security features and control over your AI development ecosystem.

---

## Our Culture

Hugging Face fosters an **open, collaborative, and inclusive** environment focused on ethical AI development. The company believes in community-driven innovation, empowering individual contributors, researchers, and enterprises alike to shape the future of AI responsibly. Transparency, knowledge sharing, and continuous learning are core values embraced by the Hugging Face ecosystem.

---

## Customers & Use Cases

Hugging Face serves a diverse audience:

- **Machine Learning Engineers & Scientists:** Research and deploy models and datasets.
- **Developers & Data Scientists:** Leverage pre-trained models and datasets to accelerate product development.
- **Enterprise Teams:** Build secure, scalable AI applications using Hugging Face’s enterprise platform.
- **Educational Institutions:** Use open datasets and models for learning and academic research.
- **AI Enthusiasts & Innovators:** Participate in a thriving community to experiment and contribute knowledge.

---

## Careers at Hugging Face

Join a pioneering AI company dedicated to shaping the future of machine learning:

- Work alongside leading AI researchers and passionate developers.
- Contribute to open-source projects with global impact.
- Thrive in a culture that values autonomy, collaboration, and innovation.
- Explore opportunities in engineering, research, product, community engagement, and enterprise solutions.

**Interested?** Check the Hugging Face careers page for open positions and join the vibrant AI revolution!

---

## Connect With Us

- Visit: [huggingface.co](https://huggingface.co)
- Join the community to collaborate on models, datasets, and applications.
- Explore trending models, datasets, and AI apps running live on our platform.
- Sign up to start building and sharing your AI projects today!

---

**Hugging Face** — The home of machine learning innovation and collaboration.

---

*Brand Colors:*  
Yellow #FFD21E, Orange #FF9D00, Gray #6B7280

*Logo Available In:* SVG, PNG, AI formats for brand consistency.

---

*Accelerate your Machine Learning with Hugging Face—Where the future of AI is built together.*